In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%%capture
import pandas as pd
import networkx as nx
pd.set_option('display.max_columns', None) # so it prints all the dataframe columns ...

!pip install demjson3
from demjson3 import decode
import json

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
nltk.download('stopwords')

In [ ]:
from pathlib import Path
import os
import glob
csv_df = pd.DataFrame(columns=[])
# for file in glob.glob("/content/drive/MyDrive/changed_CD012768*.csv"): # tested the whole thing only on "changed_CD012768.csv" and "changed_CD012768 (copy).csv" ...
for file in glob.glob("/content/drive/MyDrive/Feature_extraction_results/*.csv"):
  filename = os.path.basename(file)
  temp_df = pd.read_csv(file, index_col=0)
  temp_df[ "Filename" ]=filename
  if ( "'int'" in str(type(csv_df)) ):
    csv_df = temp_df.copy()
  else:
    csv_df = pd.concat([csv_df, temp_df], axis=0, ignore_index=True)
  print( "All " + str(temp_df.shape[0]) + " rows of \"" + file + "\" were added to \"csv_df\" .. " )
  # print( "\"csv_df\" has " + str(csv_df.shape[0]) + " rows ..." )
  # csv_df = csv_df.drop_duplicates()
  print( "\"csv_df\" has " + str(csv_df.shape[0]) + " rows after duplicate removal ..." )
  print( "===========" )

print( "Total count of of \"PMID\" values:\t" + str(csv_df.shape[0]) )
print( "Duplicate count of \"PMID\" column:\t" + str(csv_df.duplicated(subset="PMID").sum()) )

pmids = list(csv_df["PMID"])
pmids = list(dict.fromkeys(pmids)) # remove duplicate PMID values
print( "Number of distinct \"PMID\" values:\t" + str(len(pmids)) )

All 119 rows of "/content/drive/MyDrive/Feature_extraction_results/finished_new_prompt_CD012768_hard.csv" were added to "csv_df" .. 
"csv_df" has 119 rows after duplicate removal ...
All 119 rows of "/content/drive/MyDrive/Feature_extraction_results/ada_embed_CD012768.csv" were added to "csv_df" .. 
"csv_df" has 238 rows after duplicate removal ...
All 119 rows of "/content/drive/MyDrive/Feature_extraction_results/finished_new_prompt_CD012768_soft.csv" were added to "csv_df" .. 
"csv_df" has 357 rows after duplicate removal ...
All 119 rows of "/content/drive/MyDrive/Feature_extraction_results/finished_new_prompt_CD012768.csv" were added to "csv_df" .. 
"csv_df" has 476 rows after duplicate removal ...
All 1799 rows of "/content/drive/MyDrive/Feature_extraction_results/ada_embed_CD008874.csv" were added to "csv_df" .. 
"csv_df" has 2275 rows after duplicate removal ...
All 1799 rows of "/content/drive/MyDrive/Feature_extraction_results/finished_new_prompt_CD008874.csv" were added to "c

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/checkpoint_12_df.csv")
df.columns

Index(['abstract', 'authors', 'chemicals', 'citation', 'doi', 'history',
       'issn', 'issue', 'journal', 'keywords', 'mesh', 'pmc', 'pmid',
       'publication_types', 'title', 'url', 'volume', 'volume_issue', 'xml',
       'year', 'author_first', 'references', 'issn_data'],
      dtype='object')

In [ ]:
# this converts checkpoint_11 to checkpoint_12

import requests
from IPython.display import clear_output

def get_title(row):
  try:
    references_json = decode(row["references"])
    new_ref=[]
    for ref in references_json:
      # clear_output(wait=True)
      # print( ref )
      title = ref["citation"]
      arr = title.split()
      if ( len(arr) == 1 ):
        clear_output(wait=True)
        print( row["references"] )
        print(title)
        response = requests.get(f'https://api.openalex.org/works/doi:{title}')
        decoded_json = decode(response.text)
        authors = ""
        for author in decoded_json["authorships"]:
          authors = authors + author["raw_author_name"] + ", "
        authors = authors[:-2]
        new_cit = authors + " (" + str(decoded_json["publication_year"]) + ") " + decoded_json["title"] + " " + title
        new_json = {}
        new_json["citation"] = new_cit
        new_json["doi"] = title
        new_ref.append(new_json)
      else:
        new_ref.append(ref)
    return json.dumps(new_ref)

  except:
    return row["references"] # only gets here if "Page Not Found"

# ref_arr = csv_df[csv_df["Filename"] == "finished_new_prompt_CD012768_soft.csv"].apply( get_title, axis=1 )
# df = pd.read_csv("/content/drive/MyDrive/checkpoint_11_df.csv")
df = df.astype(str)
df["references"] = df.apply( get_title, axis=1 )

In [ ]:
df.to_csv("/content/drive/MyDrive/checkpoint_12_df.csv", encoding='utf-8', index=False)

In [ ]:
csv_df = csv_df.astype(str)

In [ ]:
no_ref_positive = csv_df[ (csv_df["references"] == "[]") & (csv_df["Status"] == "1") ]
no_ref_negative = csv_df[ (csv_df["references"] == "[]") & (csv_df["Status"] == "0") ]
print(f'There are {len(no_ref_positive)} postive articles with no references and {len(no_ref_negative)} negative articles with no references.')
# no_ref_positive["Filename"].value_counts() # prints how many positive no ref per file
# no_ref_negative["Filename"].value_counts() # prints how many negative no ref per file

There are 254 postive articles with no references and 22806 negative articles with no references.


In [ ]:
csv_df.shape

(105521, 56)

In [ ]:
csv_df.columns

Index(['Criteria Similarity', 'PMID', 'Review Criteria', 'Review Title',
       'Status', 'Title', 'Topic', 'ada_embedding', 'author_first', 'authors',
       'c_score', 'chemicals', 'citation', 'doi', 'f_score', 'hard1', 'hard2',
       'hard3', 'hard4', 'hard5', 'history', 'issn', 'issn_data', 'issue',
       'journal', 'keywords', 'mesh', 'pmc', 'prediction', 'publication_types',
       'query1', 'query2', 'query3', 'query4', 'query5', 'question1',
       'question2', 'question3', 'question4', 'question5', 'rank',
       'references', 'score', 'soft1', 'soft2', 'soft3', 'soft4', 'soft5',
       'text', 'url', 'vector1', 'volume', 'volume_issue', 'xml', 'year',
       'Filename'],
      dtype='object')

In [ ]:
csv_df_copy = csv_df.copy(deep=True)

In [ ]:
csv_df = csv_df_copy.copy(deep=True)

In [ ]:
csv_df["references"].describe()

,references
count,105521
unique,20943
top,[]
freq,23091


In [ ]:
csv_df["Filename"].unique()

array(['finished_new_prompt_CD012768_soft.csv',
       'finished_new_prompt_CD012768_hard.csv', 'ada_embed_CD012768.csv',
       'finished_new_prompt_CD012768.csv', 'ada_embed_CD008874.csv',
       'finished_new_prompt_CD008874.csv',
       'finished_new_prompt_CD008874_soft.csv', 'ada_embed_CD009044.csv',
       'finished_new_prompt_CD009044.csv',
       'finished_new_prompt_CD009044_hard.csv',
       'finished_new_prompt_CD009044_soft.csv', 'ada_embed_CD011686.csv',
       'finished_new_prompt_CD011686.csv',
       'finished_new_prompt_CD011686_hard.csv',
       'finished_new_prompt_CD011686_soft.csv', 'ada_embed_CD012080.csv',
       'finished_new_prompt_CD012080.csv',
       'finished_new_prompt_CD012080_hard.csv',
       'finished_new_prompt_CD012080_soft.csv',
       'finished_new_prompt_CD012233_soft.csv',
       'finished_new_prompt_CD012233.csv',
       'finished_new_prompt_CD012233_hard.csv', 'ada_embed_CD012233.csv',
       'ada_embed_CD012567.csv', 'finished_new_prompt_CD01

In [ ]:
csv_df[csv_df["Filename"] == "finished_new_prompt_CD012768_soft.csv"]["references"]

,references
0,"[{""doi"": ""10.1164/ajrccm/148.5.1292"", ""citatio..."
1,"[{""doi"": ""10.4103/0019-5413.93688"", ""citation""..."
2,"[{""doi"": ""10.5.3.327"", ""citation"": ""Expert Rev..."
4,"[{""doi"": ""10.1128/CMR.00042-10"", ""citation"": ""..."
5,"[{""doi"": ""10.1016/j.tube.2013.04"", ""citation"":..."
...,...
112,"[{""citation"": ""WHO Health Topics: Meningitis...."
113,"[{""doi"": ""10.2217/fmb.11"", ""citation"": ""Future..."
114,"[{""citation"": ""Geneva, Switzerland: World Heal..."
115,"[{""doi"": ""10.1128/JCM.00491-11"", ""citation"": ""..."


In [ ]:
csv_df.shape

(105521, 56)

In [ ]:
def get_empty(row):
  if ( row["references"] == "[]" ):
    return row["PMID"]
  if ( str(row["references"]) == "nan" ):
    return row["PMID"]
  return ""

empty_row_iter = csv_df.apply( get_empty, axis=1 )
empty_row_iter = [x for x in empty_row_iter if x != ""]
csv_df = csv_df[~csv_df["PMID"].isin(empty_row_iter) ] # here ~ is equivalent to NOT (negates the statement)

In [ ]:
csv_df.shape

(82426, 56)

In [ ]:
len(empty_row_iter)

23095

In [ ]:
# you might have the same reference formulated differently amongst more rows ...
# since there are references that have no unique identifier (doi/pmid) maybe it would be best
# if i did an NLP comparison between em to see if they are the same ...
# this would then be a citation network that is based on citation SIMMILARITY not identicality which might NOT have been done yet ...
# !pip install fix-busted-json
# from fix_busted_json import repair_json

import json
!pip install demjson3
from demjson3 import decode
from IPython.display import clear_output

missed = []
def get_json_citation(row):
  global missed
  succeeded = True

  try:
    references_json = decode(row["references"])
    citations=[]
    for ref in references_json:
      # clear_output(wait=True)
      # print( ref )
      citations.append( ref["citation"] )
    return json.dumps(citations)
  except:
    missed.append( row["references"] )

# if ( not succeeded ):
#   try:
#     missed.append( row["references"] )
#     references_json = repair_json(row["references"])
#     citations=[]
#     for ref in references_json:
#       clear_output(wait=True)
#       print( ref )
#       citations.append( ref["citation"] )
#     return json.dumps(citations)
#   except:
#     missed.append( row["references"] )
#     clear_output(wait=True)
#     print(len(missed))
#     return "[]"

# print(missed)

filenames = csv_df["Filename"].unique().tolist()
cit_dfs = []
all_citations = []
for filename in filenames:
  temp_df = csv_df[ csv_df["Filename"] == filename ].copy(deep=True)
  print( f'temp: {temp_df.shape}' )
  temp_df["reference_array"] = temp_df.apply( get_json_citation, axis=1 )
  all_citations.extend( temp_df["reference_array"].tolist() )
  file_df = temp_df[["PMID", "citation", "reference_array", "Status"]]
  cit_dfs.append(file_df)

# print(cit_dfs)

temp: (106, 56)
temp: (106, 56)
temp: (106, 56)
temp: (106, 56)
temp: (1450, 56)
temp: (1450, 56)
temp: (1450, 56)
temp: (3014, 56)
temp: (3014, 56)
temp: (3014, 56)
temp: (3014, 56)
temp: (5974, 56)
temp: (5974, 56)
temp: (5974, 56)
temp: (5974, 56)
temp: (5008, 56)
temp: (5008, 56)
temp: (5008, 56)
temp: (5008, 56)
temp: (282, 56)
temp: (282, 56)
temp: (282, 56)
temp: (282, 56)
temp: (4148, 56)
temp: (4148, 56)
temp: (4148, 56)
temp: (4148, 56)
temp: (987, 56)
temp: (987, 56)
temp: (987, 56)
temp: (987, 56)


In [ ]:
csv_df_copy_1 = csv_df.copy(deep=True)

In [ ]:
cit_dfs_copy = []

for i in range(0, len(cit_dfs)):
  cit_dfs_copy.append(cit_dfs[i].copy(deep=True))

In [ ]:
cit_dfs = []

for i in range(0, len(cit_dfs_copy)):
  cit_dfs.append(cit_dfs_copy[i].copy(deep=True))

In [ ]:
cit_dfs[0]

In [ ]:
cit_dfs[0][cit_dfs[0]["Status"] == "1"]

In [ ]:
cit_dfs0 = cit_dfs[0].copy(deep=True)
cit_dfs0

In [ ]:
cit_dfs[0] = cit_dfs_copy[0].copy(deep=True)

In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt') # if necessary...
nltk.download('punkt_tab')

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    # print( tfidf )
    # return ((tfidf * tfidf.T).A)[0,1]
    return ((tfidf * tfidf.T))[0,1]


cosine_sim('PLoS One Claassens 8 e76272 2013 10.1371/journal.pone.0076272 Tuberculosis in healthcare workers and infection control measures at primary healthcare facilities in South Africa', 'van Soolingen D, Hermans PW, de Haas PE, Soll DR, van Embden JD. Occurrence and stability of insertion sequences in Mycobacterium tuberculosis complex strains: evaluation of an insertion sequence-dependent DNA polymorphism as a tool in the epidemiology of tuberculosis. Journal of clinical microbiology. 1991;29(11):2578–86.')

0.039212891527794676

In [ ]:
len(cit_dfs)

31

In [ ]:
cit_dfs[0]["citation"]

,citation
0,"Peter JG, et al. The diagnostic accuracy of ur..."
1,"Held M, et al. Diagnostic Accuracy of the Xper..."
2,"Chang K, et al. Rapid and effective diagnosis ..."
4,"Patel VB, et al. Diagnostic accuracy of quanti..."
5,"Hodkinson B, et al. HIV Infection and Osteoart..."
...,...
112,"Rajasingham R, et al. Epidemiology of meningit..."
113,Lawn SD and Zumla AI. Diagnosis of extrapulmon...
114,"Du J, et al. Rapid diagnosis of pleural tuberc..."
115,"Marouane C, et al. Evaluation of molecular det..."


In [ ]:
import os
directory="/content/drive/MyDrive/Citation_dataframes"
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
stop_words = set(stopwords.words('english'))

all_refs = []

def ensure_positive(a):
  if ( a < 0 ):
    return -a
  else:
    return a

def tokenise_citation(cit):
  global stop_words
  tokenizer = RegexpTokenizer(r'\w+') # tokenises after removing punctuation marks
  word_tokens = tokenizer.tokenize(cit)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] # remove stopwords
  return filtered_sentence

def get_common_citation_word_count(cit1, cit2):
  cit1_tokens = tokenise_citation(cit1)
  cit2_tokens = tokenise_citation(cit2)
  # cit1_tokens = ["0", "7", "6", "5", "4", "3", "2"]
  # cit2_tokens = ["1", "2", "9", "8", "7", "6", "5"]
  # common_tokens = list(set(cit1_tokens) & set(cit2_tokens))
  # common_tokens # outputs ['7', '5', '2', '6'], it's ok
  common_tokens = list(set(cit1_tokens) & set(cit2_tokens)) # this compares unique words, each "set" contains NO duplicate words ...
  return len(common_tokens)

def check_if_viable_for_cosine_sim(cit, is_positive=False):
  global all_refs
  global skipped
  global applied
  global df_rows
  global row_iter
  global cit_iter
  global average_citation_count
  global df_cit_iter
  should_cosine=False

  iter=0
  refs_length = len(all_refs)
  arr=[]
  skip_count = 0

  start_iter = -1
  end_iter = -1

  if ( is_positive == False ):
    if ( df_rows <= 1500 ):
      for i in range(0, refs_length):
        if ( round(len(cit)/len(all_refs[i]), 3) <= 1.000 ): # if it's < 1.00 is almost right, but it cuts some rows ...
          if ( start_iter == -1 ):
            start_iter = i
            continue
        if ( round(len(cit)/len(all_refs[i]), 3) <= 0.995 ):
          if ( end_iter == -1 ):
            end_iter = i + 1
            break

    if ( df_rows >= 1500 ):
      for i in range(0, refs_length):
        if ( round(len(cit)/len(all_refs[i]), 8) <= 1.00000000 ): # if it's < 1.00 is almost right, but it cuts some rows ...
          if ( start_iter == -1 ):
            start_iter = i
            continue
        if ( round(len(cit)/len(all_refs[i]), 8) <= 0.99999995 ):
          if ( end_iter == -1 ):
            end_iter = i + 1
            break

    skip_count = refs_length - end_iter + start_iter
    cit_num = end_iter - start_iter + 1
  else:
    start_iter = 0
    end_iter = refs_length
    cit_num = refs_length


  for i in range(start_iter, end_iter):
    if ( len(arr) == int(average_citation_count) and is_positive == True ):
      break;

    if ( all_refs[i] == cit ): # there won't be any (or nearly enough) if you search them to be identical ...
      skip_count = skip_count + 1
      continue

    # if ( ensure_positive( len(cit) - len(u) ) > 10 ):
    #   # print(ensure_positive( len(cit) - len(u) ))
    #   skip_count = skip_count + 1
    #   continue

    wordcount = get_common_citation_word_count( cit, all_refs[i] )
    # iter = iter + 1
    # print( f'Going though ref {iter} / {refs_length} ...' )
    treshold = 5
    if ( df_rows >= 1500 ):
      treshold = 7
    if ( df_rows >= 5000 ):
      treshold = 10
    if ( wordcount >= treshold ): # adjust the treshold here ...
      clear_output(wait=True)
      arr.append(all_refs[i])
      print(f'Computing row {row_iter} / {df_rows}: comparing citation {cit_iter} with citation {i - start_iter + 1} / {cit_num} (out of {refs_length} citations of the current df) ... \nstart_iter={start_iter} end_iter={end_iter}\nApplying cosine simmilarity wordcount={wordcount} ... \nDid this for {applied} citations so far ... skipped {skipped} citations so far ... ')
      applied = applied + 1
      # time.sleep(1)
      # should_cosine=True
      # break
    else:
      skip_count = skip_count + 1

  if ( len(arr) == 0 and is_positive == True ):
    # print( 1 + "1" )
    for i in range(0, refs_length):
      if ( ensure_positive( len(cit) - len(all_refs[i]) ) < 2 ):
        if ( start_iter == -1 ):
          start_iter = i
      if ( ensure_positive( len(cit) - len(all_refs[i]) ) > 10 ):
        if ( end_iter == -1 ):
          end_iter = i + 1
          break
    skip_count = refs_length - end_iter + start_iter
    cit_num = end_iter - start_iter + 1

    for i in range(start_iter, end_iter):
      if ( i > int(average_citation_count) ):
        break;
      arr.append(all_refs[i])

  if ( len(arr) == 0 and is_positive == True ):
    print( 1 + "1" )
    import random
    for i in range(0, int(average_citation_count)):
      rand = random.choice(all_refs)
      if ( rand in arr ):
        i = i - 1
        continue
      arr.append(all_refs[i])

  # if ( len(arr) > average_citation_count ): # for maximum citation count
  #   average_citation_count = len(arr)

  average_citation_count = ( average_citation_count * df_cit_iter + len(arr) ) / ( df_cit_iter + 1 )
  df_cit_iter = df_cit_iter + 1

  if (len(arr)!=0):
    return arr
  else:
    clear_output(wait=True)
    skipped = skipped + skip_count
    print(f'Computed row {row_iter} / {df_rows}.\nSkipped {skipped} citations, applied cosine simmilarity to {applied} citations so far ... ')
    return []

def get_cosine_simmilarity(ref, arr):
  global all_refs
  count = 1
  cosines=[]
  for u in arr:
    temp_json = {}
    temp_json["citation"] = u.rstrip()
    temp_json["cosine"] = cosine_sim( ref, u )
    temp_json["id"] = all_refs.index(u)
    cosines.append(temp_json) # could average them when decide which to include in the citation network ...
  return cosines

def get_refs(row):
  global all_refs
  ref_json = decode(row["reference_array"])
  for ref in ref_json:
    if ( ref not in all_refs ):
      all_refs.append(ref)
  # print(1+"1")
  all_refs.append(row["citation"])
  all_refs = sorted(all_refs, key=len)

total_cosine = 0
def get_ref_freq(row):
  global all_refs
  global row_iter
  global cit_iter
  global total_cosine

  is_positive = False
  if ( str(row["Status"]) == "1" ):
    is_positive = True

  row_iter = row_iter + 1
  ref_json = decode(row["reference_array"])
  freq = []
  cit_iter = 0
  for ref in ref_json:
    cit_iter = cit_iter + 1

    arr = check_if_viable_for_cosine_sim(ref, is_positive)
    if ( arr != [] ):
      total_cosine = total_cosine + 1
      # print(1+"1")
      # freq.append( '{ citation:"' + ref + '", freq:' + str(all_refs.count(ref)) + ", id:" + str(all_refs_unique.index(ref)) + "}")
      temp_json = {}
      temp_json["citation"] = ref.rstrip()
      # temp_json["freq"] = str(all_refs.count(ref))
      temp_json["cosine"] = get_cosine_simmilarity(ref, arr)
      temp_json["id"] = all_refs.index(ref)
      freq.append( temp_json )
  return freq

start = time.time()
for i in range(10, len(cit_dfs)):
# for i in range(0, 1):
  # if ( i == 6 ):
  #   continue
  all_refs = []
  skipped = 0
  applied = 0
  df_rows = len(cit_dfs[i])
  row_iter = 0
  cit_iter = 0
  df_cit_iter = 0
  average_citation_count = 0
  print("Getting refs for current citation df ...", end="")
  cit_dfs[i] = cit_dfs[i].sort_values("Status")
  cit_dfs[i].apply( get_refs, axis=1 )
  print(" - DONE")
  cit_dfs[i]["reference_array"] = cit_dfs[i].apply( get_ref_freq, axis=1 )

  cit_dfs[i].to_csv("/content/drive/MyDrive/Citation_dataframes/cit_dfs_" + str(i) + "_" + filenames[i], encoding='utf-8', index=False)

end = time.time()
length = end - start

print(length)
print(length/60)
print(length%60)
# cit_dfs[0]
# ok
# SHOULD DO SOMETHING TO ENSURE THAT FOR THE POSITIVE ARTICLES ALL CITATIONS ARE CONSIDERED ... THAT THEY DONT HAVE AN EMPTY CITATION LISTN AFTER THIS ...

Computed row 2776 / 3014.
Skipped 3818422252 citations, applied cosine simmilarity to 210104 citations so far ... 


In [ ]:
cit_dfs = []

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from pathlib import Path
import os
import glob
csv_df = pd.DataFrame(columns=[])
for file in glob.glob("/content/drive/MyDrive/Citation_dataframes/*.csv"):
  filename = os.path.basename(file)
  temp_df = pd.read_csv(file, index_col=0)
  cit_dfs.append(temp_df)

Mounted at /content/drive


In [ ]:
# this is just to ensure the validity of the jsons inside "reference_array"

def get_json(row):
  res_json = decode(row["reference_array"])
  # print( res_json )

for i in range(0, len(cit_dfs)):
  cit_dfs[i].apply( get_json, axis=1 )
  # cit_dfs[i] = cit_dfs[i].astype(str)

In [ ]:
total_cosine

662

In [ ]:
round(23/18 - 1)

0

In [ ]:
filenames

['finished_new_prompt_CD012768_hard.csv',
 'ada_embed_CD012768.csv',
 'finished_new_prompt_CD012768_soft.csv',
 'finished_new_prompt_CD012768.csv',
 'ada_embed_CD008874.csv',
 'finished_new_prompt_CD008874.csv',
 'finished_new_prompt_CD008874_soft.csv',
 'ada_embed_CD009044.csv',
 'finished_new_prompt_CD009044.csv',
 'finished_new_prompt_CD009044_hard.csv',
 'finished_new_prompt_CD009044_soft.csv',
 'ada_embed_CD011686.csv',
 'finished_new_prompt_CD011686.csv',
 'finished_new_prompt_CD011686_hard.csv',
 'finished_new_prompt_CD011686_soft.csv',
 'ada_embed_CD012080.csv',
 'finished_new_prompt_CD012080.csv',
 'finished_new_prompt_CD012080_hard.csv',
 'finished_new_prompt_CD012080_soft.csv',
 'finished_new_prompt_CD012233_soft.csv',
 'finished_new_prompt_CD012233.csv',
 'ada_embed_CD012233.csv',
 'finished_new_prompt_CD012233_hard.csv',
 'ada_embed_CD012567.csv',
 'finished_new_prompt_CD012567.csv',
 'finished_new_prompt_CD012567_hard.csv',
 'finished_new_prompt_CD012567_soft.csv',
 'ada_

In [ ]:
cit_dfs[0]["reference_array"].describe()

,reference_array
count,106
unique,67
top,[]
freq,40


In [ ]:
cit_dfs[0].to_csv("/content/drive/MyDrive/cit_dfs0_just_cosine_simmilarity_1.csv", encoding='utf-8', index=False)

['7', '5', '2', '6']

In [ ]:
cit_dfs[0] = cit_dfs[0].astype(str)

def print_refs(row):
  global all_refs
  ref_json = decode(row["reference_array"])
  for ref in ref_json:
    # print(ref)
    # ref = decode(ref)
    print(ref["id"])
    print(ref["citation"])
    print(ref["freq"])
    if ( str(ref["freq"]) != "1" ): # they are ALL 1 ... so, I would want to make a NLP-type comparison of them, to have a frequency that is NOT 1 ...
      print( "NOT 1" )
      print(1 + "1")
    print("\n\n")

cit_dfs[0].apply( print_refs, axis=1 )

1539
The American review of respiratory disease BE Jones 148 1292 1993 10.1164/ajrccm/148.5.1292 Relationship of the manifestations of tuberculosis to CD4 cell counts in patients with human immunodeficiency virus infection.
313
NOT 1


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
import json
data = {"pmid": [], "citation": []}
citation_df = pd.DataFrame(data)

def parse_citations(row):
  references_json = json.loads(row["references"])
  for ref in references_json:
    if ( "pubmed" in ref["article_ids"] ):
      citation_df.loc[len(citation_df)] = [row["pmid"], "PMID-"+str(ref["article_ids"]["pubmed"])]
      continue
    if ( "pmc" in ref["article_ids"] ):
      citation_df.loc[len(citation_df)] = [row["pmid"], "PMC-"+str(ref["article_ids"]["pmc"])]
      continue

    citation_df.loc[len(citation_df)] = [row["pmid"], "TITLE-"+str(ref["citation"])]
    continue

citations.apply( parse_citations, axis=1 )
citation_df = citation_df.astype(str)
citation_df

In [ ]:
citation_df.describe()

In [ ]:
# Citation network with ALL citations ...

citation_network = nx.from_pandas_edgelist(citation_df, "pmid", "citation")
# nx.draw_spring(citation_network) # draw_planar / draw / draw_spring - those are the best visualisation functions ...
print( "Number of connected components: " + str( nx.number_connected_components(citation_network) ) )

In [ ]:
# CODE FOR MAKING A CITATION NETWORK FOR EACH ARTICLE INDIVIDUALLY

# initial_article_pmids = list(citation_df['pmid'].unique())

# # article_dfs=[]
# for initial_article_pmid in initial_article_pmids:
#   article_df = citation_df[citation_df["pmid"]==initial_article_pmid]

#   citation_network = nx.from_pandas_edgelist(article_df, "pmid", "citation")
#   nx.draw_spring(citation_network) # draw_planar / draw / draw_spring - those are the best visualisation functions ...
#   print( "Number of connected components: " + str( nx.number_connected_components(citation_network) ) )

#   # article_dfs.append( article_df )

# article_df